Implement Deep Convolutional Generative Adversarial Networks (DCGAN) in Keras

In [0]:
import keras
from keras import layers
import numpy as np
import os
from keras.preprocessing import image

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


### Generator Network 

The Generator model turns a vector from the latent space into a candidate image. We use dropout to prevent the generator from getting stuck with generated images that look like noise


In [0]:
latent_dim = 32
height = 32
width = 32
channels = 3

generator_input = keras.Input(shape=(latent_dim,))

#Transform the input into a 16 x 16 128 channel feature map
x = layers.Dense(128*16*16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16,16,128))(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

#Upsample to 32 x 32
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

#Produces a 32x32 1-channel feature map (shape of a CIFAR10 image)
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)

#Instantiates the generator model, which maps the input of shape (latent_dim) into an image of shape (32,32,3)
generator = keras.models.Model(generator_input, x)
generator.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv

### Discriminator Network 

A discriminator model takes as input a candidate image (real or synthetic) and classifies it into one of two classes: “generated image” or “real image that comes from the training set.”

In [0]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

#One dropout layer
x = layers.Dropout(0.4)(x)

#Classification layer
x = layers.Dense(1, activation='sigmoid')(x)

#Instantiates the discriminator model, which turns a (32, 32, 3) input into a binary classifi-cation decision (fake/real)
discriminator = keras.models.Model(discriminator_input,x)
discriminator.summary()

# Use gradient clipping. Use learning rate decay to stabilize training
discriminator_optimizer = keras.optimizers.RMSprop(lr = 0.0008, clipvalue = 1.0, decay=1e-8)

discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 6, 6, 128)         262272    
___________________________

### Adversarial Network

Now we setup the GAN by chaining the Generator network with the Discriminator network. The training process only updates the weights of the generator in a way that makes the discriminator more likely to predict "real" when looking at fake images. The weights of the discriminator are frozen i.e they are not updated during training. Otherwise we would end up training the discriminator to always predict "real" which is not our goal


In [0]:
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

### Training the DCGAN 

For each epoch :

1. Draw random points in the latent space (random noise).
2. Generate images with `generator` using this random noise.
3. Mix the generated images with real ones
4. Train `discriminator` using these mixed images, with corresponding targets:
either “real” (for the real images) or “fake” (for the generated images).
5. Draw new random points in the latent space.
6.Train `gan` using these random vectors, with targets that all say “these are real
images.” This updates the weights of the generator (only, because the discriminator
is frozen inside gan) to move them toward getting the discriminator to
predict “these are real images” for generated images: this trains the generator
to fool the discriminator.

In [0]:
(x_train, y_train), (_,_) = keras.datasets.cifar10.load_data()

#select only frog class images (class 6)
x_train = x_train[y_train.flatten() == 6] 
#Normalize the data
x_train = x_train.reshape((x_train.shape[0],) + (height, width, channels)).astype('float32')/255.

iterations = 10000
batch_size = 20
save_dir = '/content/gdrive/My Drive/Colab Notebooks/data/gan_images'

start = 0
for step in range(iterations):
  #Sample random points in the latent space
  random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
  
  #Decodes them to fake images
  generated_images = generator.predict(random_latent_vectors)
  stop = start + batch_size
  real_images = x_train[start: stop]
  
  #combine with real images
  combined_images = np.concatenate([generated_images, real_images])
  
  #assign labels, discriminating real & fake images
  labels = np.concatenate([np.ones((batch_size,1)), np.zeros((batch_size,1))])
  
  #add random noise to the labels - important trick for training
  labels += 0.05 * np.random.random(labels.shape)
  
  #Train the discriminator
  d_loss = discriminator.train_on_batch(combined_images, labels)
  
  #Samples random points in the latent space
  random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
  
  #Assemble labels that say "these are real images"
  misleading_targets = np.zeros((batch_size,1))
  
  #Train the generator (via the gan model where discriminator weights are frozen)
  a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
  
  start += batch_size
  if start > len(x_train) - batch_size:
    start = 0
    
  if step%100==0: 
    gan.save_weights('gan.h5')
    
    print('Discriminator loss:', d_loss)
    print('Adversarial loss:', a_loss)
    
    img = image.array_to_img(generated_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir,'generated_frog' + str(step) + '.png'))
    
    img = image.array_to_img(real_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir,'real_frog' + str(step) + '.png'))

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Discriminator loss: 5.2066708
Adversarial loss: 15.942385
Discriminator loss: 0.56745374
Adversarial loss: 1.2648748
Discriminator loss: 0.69516265
Adversarial loss: 0.77763397
Discriminator loss: 0.70561326
Adversarial loss: 0.74772215
Discriminator loss: 0.6780567
Adversarial loss: 0.7215992
Discriminator loss: 0.7011272
Adversarial loss: 0.75191534
Discriminator loss: 0.7009509
Adversarial loss: 0.73629373
Discriminator loss: 0.69797486
Adversarial loss: 0.78167474
Discriminator loss: 0.7018818
Adversarial loss: 0.7335743
Discriminator loss: 0.6980766
Adversarial loss: 0.7550539
Discriminator loss: 0.7686324
Adversarial loss: 0.75139606
Discriminator loss: 0.70752615
Adversarial loss: 0.72950137
Discriminator loss: 0.7012187
Adversarial loss: 0.7445418
Discriminator loss: 0.68844926
Adversarial loss: 0.76178026
Discriminator loss: 0.71396434
Adversarial loss: 0.7601091
Discriminator loss: 0.70142764
Adversarial loss: 0.7624836
Discriminator loss: 0.68898326
Adversarial loss: 0.75012